In [1]:
%pip install pandas python-dotenv requests

In [ ]:
import os
import pandas as pd
from io import StringIO
from dotenv import load_dotenv, find_dotenv

In [3]:
# Load environment variables
_ = load_dotenv(find_dotenv())

# Get OpenRouter API key and model from environment variables
openrouter_api_key = os.environ.get("OPENROUTER_API_KEY")
openrouter_model = os.environ.get("OPENROUTER_MODEL", "openai/gpt-3.5-turbo")

print(f"Using model: {openrouter_model}")

Using model: openai/gpt-3.5-turbo


In [4]:
# Prompt to generate synthetic data
prompt = """
Generate 10 rows of synthetic data for sentiment classification.
Each row should have two columns: "text" (a sentence) and "label" (positive or negative).
Return the data as a CSV format with headers: text,label.
"""

In [ ]:
# Using OpenRouter API instead of OpenAI
import requests


# OpenRouter API endpoint
url = "https://openrouter.ai/api/v1/chat/completions"

# Headers
headers = {
    "Authorization": f"Bearer {openrouter_api_key}",
    "Content-Type": "application/json"
}

# Request body
data = {
    "model": openrouter_model,
    "messages": [{"role": "user", "content": prompt}],
    "temperature": 0.7
}

# Make the API call
response = requests.post(url, headers=headers, json=data)
response_data = response.json()

In [6]:
# Extract the content from the response
csv_text = response_data['choices'][0]['message']['content']

In [7]:
csv_text

"text,label\nI love spending time with my family,positive\nThe food at that restaurant was terrible,negative\nI feel happy when I'm surrounded by nature,positive\nThe customer service at that store was amazing,positive\nI can't stand being stuck in traffic,negative\nWatching the sunset always puts me in a good mood,positive\nThe movie I watched last night was so boring,negative\nI am grateful for all the opportunities in my life,positive\nI had a terrible day at work today,negative\nI feel so lucky to have such supportive friends,positive"

In [8]:
df = pd.read_csv(StringIO(csv_text))

In [9]:
df

,text,label
0,I love spending time with my family,positive
1,The food at that restaurant was terrible,negative
2,I feel happy when I'm surrounded by nature,positive
3,The customer service at that store was amazing,positive
4,I can't stand being stuck in traffic,negative
5,Watching the sunset always puts me in a good mood,positive
6,The movie I watched last night was so boring,negative
7,I am grateful for all the opportunities in my ...,positive
8,I had a terrible day at work today,negative
9,I feel so lucky to have such supportive friends,positive


In [10]:
ner_prompt = """
Generate 5 sentences with entities in format:
{"text": "sentence here", "entities": [{"word": "entity", "label": "ORG"}, ...]}
Entities should be PERSON, ORG, LOCATION only.
"""

In [11]:
# Using OpenRouter API for NER task
# Headers remain the same as before

# Request body for NER task
ner_data = {
    "model": openrouter_model,
    "messages": [{"role": "user", "content": ner_prompt}],
    "temperature": 0.7
}

# Make the API call
ner_response = requests.post(url, headers=headers, json=ner_data)
ner_response_data = ner_response.json()

In [12]:
# Extract the NER content from the response
ner_text = ner_response_data['choices'][0]['message']['content']
print(ner_text)

1. {"text": "John works at Apple Inc. in Cupertino, California.", "entities": [{"word": "John", "label": "PERSON"}, {"word": "Apple Inc.", "label": "ORG"}, {"word": "Cupertino, California", "label": "LOCATION"}]}
2. {"text": "The CEO of Microsoft, Satya Nadella, will be speaking at the conference in Seattle next week.", "entities": [{"word": "Satya Nadella", "label": "PERSON"}, {"word": "Microsoft", "label": "ORG"}, {"word": "Seattle", "label": "LOCATION"}]}
3. {"text": "Sarah is a designer at Google in Mountain View, California.", "entities": [{"word": "Sarah", "label": "PERSON"}, {"word": "Google", "label": "ORG"}, {"word": "Mountain View, California", "label": "LOCATION"}]}
4. {"text": "The mayor of New York City, Bill de Blasio, announced a new initiative to improve public transportation.", "entities": [{"word": "Bill de Blasio", "label": "PERSON"}, {"word": "New York City", "label": "LOCATION"}]}
5. {"text": "Mark Zuckerberg founded Facebook in Menlo Park, California.", "entities"